# pandas 練習  


## pandas とは  

> プログラミング言語Pythonにおいて、データ解析を支援する機能を提供するライブラリである。特に、数表および時系列データを操作するためのデータ構造と演算を提供する。([Wikipedia](https://ja.wikipedia.org/wiki/Pandas)より)  




---

In [ ]:
# モジュールのインポート
import pandas as pd
import numpy as np  

以降の説明のために簡単な人工データを用意します。  

pandas の `DataFrame` クラスを作成します。5 行 4 列から成る乱数の配列と、列の見出し (ラベル) として `A - D` のアルファベットを指定しています。  

In [ ]:
np.random.seed(0)
df_random_values = pd.DataFrame(np.random.rand(5, 4), columns=["A", "B", "C", "D"])
df_random_values

In [ ]:
# クラスの確認
type(df_random_values)

## データの参照と抽出  

### ラベルを指定して列を抽出  

`DataFrame["label"]` で指定したラベルの列を抽出することができます。  

In [ ]:
df_random_values["A"]

このような 1 列のみのデータは `Series` というクラスになります。

In [ ]:
# クラスの確認
type(df_random_values["A"])

複数のラベルをリストとして指定することも可能です。(この場合のクラスは DataFrame)  

In [ ]:
df_random_values[["A", "D"]]

### ラベルによる指定の loc() メソッドと位置による指定の iloc() メソッド  

DataFrame クラスの持つ `loc()` メソッドを使って、行と列を指定して次のような抽出ができます。  
この `[ ]` の中の `2:4` はインデックスが 2 ～ 4 であることを意味し、`["B", "C"]` は B 列と C 列を意味します。このように `loc()` だと DataFrame クラスに付与されたインデックスと列名で指定します。  

In [ ]:
df_random_values.loc[2:4, ["B", "C"]]

次の例のように `iloc()` メソッドを使って数値で指定することもできます。  
`loc()` のときの `2:4` は抽出される行のインデックスが 2 ～ 4 だったのに対し、`iloc()` での `2:4` は 2 ～ 3 のみになっていることに注意してください。`iloc()` の仕様はリストや配列でのインデックス指定と同じになっています。  

このように `loc()` はラベルによる指定なのに対し、`iloc()` は位置による指定になってます。  

In [ ]:
df_random_values.iloc[2:4, 1:3]

### ひとつのセルを指定する at() メソッドと iat() メソッド  

ひとつのセルを指定して、値を抽出したり、書き換えたりする用途としては、`at(), iat()` メソッドがあります。  
両者の違いは `loc(), iloc()` と同様に、指定するのがラベル/ラベルのインデックスかどうかです。  

In [ ]:
df_random_values.at[0, "A"]

In [ ]:
df_random_values.iat[0, 0]

### 数値との比較による抽出  

DataFrame に、真偽値を要素とし DataFrame の行数と同じ要素数の Series やリストを与えると、`True` に対応する行のみを抽出することができます。  

In [ ]:
sr_bool = pd.Series([True, False, True, False, True])
df_random_values[sr_bool]

また、DataFrame に対して比較演算子を使って数値と比較すると…  

In [ ]:
df_random_values <= 0.5

このように、真偽値 (条件を満足すれば `True`、そうでなければ `False`) を要素にした同じサイズの DataFrame が返ります。  

Series と比較すると 同様の Series が返ります。    

In [ ]:
df_random_values["A"] <= 0.5

In [ ]:
# クラスの確認
type(df_random_values["A"] <= 0.5)

これらを組み合わせると、**DataFrame の特定の列の値と数値とを比較し、条件に合った値を持つ行を抽出することができます。**  

In [ ]:
df_random_values[df_random_values["A"] <= 0.5]

複数の条件で抽出するときは次のように `&` (AND) や `|` (OR) などの演算子を用います。`~` (NOT) も使えます。  

In [ ]:
df_random_values[(df_random_values["A"] <= 0.5) & (df_random_values["C"] > 0.7)]

In [ ]:
df_random_values[(df_random_values["A"] <= 0.5) | (df_random_values["C"] > 0.7)]

In [ ]:
df_random_values[(df_random_values["A"] <= 0.5) & ~(df_random_values["C"] > 0.7)]

文字列を要素とする列と文字列を比較する場合は、以下のようにします。  

```
df[df["str_columns"]=="string"]  # 一致する場合
df[df["str_columns"]!="string"]  # 一致しない場合
```

## データの加工  

### 列の追加、上書き  

既存の DataFrame 型に対して何らかの計算を行ってデータを付加したいことがよくあります。試しに、`C` 列と `D` 列を足し算したものを `E` 列として追加してみましょう。  

In [ ]:
# 新しい列ラベルを指定して追加
df_random_values["E"] = df_random_values["C"] + df_random_values["C"]
df_random_values

このようにして列を増やすことができますが、同じような書き方で列を上書きすることもできます。例として、`B` 列を 2 倍したものを `A` 列に上書きしてみましょう。  

In [ ]:
# 既存の列ラベルを指定して上書き
df_random_values["A"] = df_random_values["B"] * 2
df_random_values

### 値の上書き  

`at(), iat()` メソッドでセルを指定して値を上書きすることもできます。  

In [ ]:
df_random_values.at[1, "A"] = -1
df_random_values

In [ ]:
df_random_values.iat[2, 0] = -1
df_random_values

## 欠損値の処理  

データ分析でよく問題になるのが欠損値の処理です。ここで以下の説明のためにわざと欠損値を入れてみます。  
なお、`None` は Python において、値が存在しないことを表すデータ型です。さらに `None` は 、pandas では `NaN` として表示されます。  

In [ ]:
df_random_values["F"] = [1, None, None, 1, None]
df_random_values

### 欠損値を無視する dropna() メソッド  

`dropna()` メソッドによって欠損値を無視してデータを分析を進めることができます。  

In [ ]:
df_random_values.dropna()

### 欠損値を特定の値で書き換える fillna() メソッド  

`fillna()` メソッドで欠損値のを特定の値で置き換えます。  

In [ ]:
df_random_values.fillna(0)

なお、`dropna()` も `fillna()` も、上のようなコードでは元の `df_random_values` を書き換えるものではありません (破壊的な操作ではない)。  

In [ ]:
# 前のセルの操作で元のデータに変更がないことを確認
df_random_values

以下のように新たな変数に代入することで書き換えられた DataFrame を得ることができます。  

In [ ]:
example_fillna = df_random_values.fillna(0)
example_fillna

## groupby の使い方  

特定の列の値を指定してグループ化し、一括して関数を適用させた結果を表示させることができます。  

In [ ]:
# グループのキーとするための G 列を追加
df_random_values["G"] = ["X", "X", "Y", "Y", "Z"]
df_random_values

In [ ]:
# 列の値をキーにグループ化
gr = df_random_values.groupby("G")
# gr = df.groupby("G", as_index=False)  # キーを index にしたくない場合

type(gr)

`DataFrameGroupBy` クラスに適用できる関数は以下の通りです。  

|関数名|説明|
|--- |--- |
|mean()|平均を計算します|
|sum()|合計を計算します|
|size()|グループの大きさを計算します|
|count()|グループのデータの個数を計算します|
|std()|標準偏差を計算します|
|var()|分散を計算します|
|sem()|平均値の標準誤差を計算します|
|describe()|グループ内の統計量を返します|
|first()|グループ内の先頭の値を返します|
|last()|グループ内の最後の値を返します|
|nth()|n番目の要素を返します。リストで指定することも可|
|min()|最小値を返します|
|max()|最大値を返します|


In [ ]:
# 合計値の表示
gr.sum()

## データの連結  

### DataFrame を縦に連結する concat() メソッド  

In [ ]:
# 連結するためのもうひとつの DataFrame を作成
df_to_concat = pd.DataFrame(np.random.rand(5, 4), columns=["A", "B", "C", "D"])
df_to_concat["E"] = df_to_concat["C"] + df_to_concat["C"]
df_to_concat["A"] = df_to_concat["B"] * 2
df_to_concat["F"] = [1, 1, None, None, None]
df_to_concat["G"] = ["Y", "M", "Y", "X", "Z"]
df_to_concat

In [ ]:
df_concat = pd.concat([df_random_values, df_to_concat])
# df_concat = pd.concat([df_random_values, df_to_concat], ignore_index=True)  # 連結した際に元の index を無視して新しく振り直す場合
df_concat

In [ ]:
df_concat.reset_index(drop=True)  # 上で ignore_index=True とした場合は不要

### DataFrame を横に連結する merge() メソッド  

In [ ]:
df_to_merge = pd.DataFrame([["X", "Suarez"], ["Y", "Messi"], ["Z", "Grieazmann"], ["XXX", "Ansu-Fati"]], columns=["G", "H"])
df_to_merge

In [ ]:
# 内部結合 (キーでソートされる)
pd.merge(df_concat, df_to_merge, on="G", how="inner")

In [ ]:
# 左結合
pd.merge(df_concat, df_to_merge, on="G", how="left")

In [ ]:
# 右結合
pd.merge(df_concat, df_to_merge, on="G", how="right")

In [ ]:
# 外部結合
pd.merge(df_concat, df_to_merge, on="G", how="outer")

## NumPy 配列型への変換  

NumPy が提供する各種演算などを利用するために、NumPy の配列型に変換することもできます。以下の例では DataFrame 型の持つメソッドのひとつである `to_numpy()` を使い、新しい変数に代入しています。

In [ ]:
numpy_array_type_data = df_random_values.loc[:, ["A", "B", "C"]].to_numpy()
numpy_array_type_data

In [ ]:
type(numpy_array_type_data)

---
## データの前処理_その1  

### データの読み込み  

[なんちゃって個人情報](http://kazina.com/dummy/) で入手したダミー個人データを加工していきます。

pandas の `read_csv()` 関数を使ってデータを読み込むことができます。他にも `read_json(), read_excel(), read_html()` など、様々なファイル形式からデータを得ることができます。  

扱うことのできるデータ形式とそれを読み込むための関数の一覧は、pandas の [公式ドキュメント](http://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) 内に一覧表があるので参照してください。下記はその抜粋です。

|フォーマット|データ形式|読み込み関数|書き出し関数|
|-|-|-|-|
|text|CSV|read_csv|to_csv|
|text|JSON|read_json|to_json|
|text|HTML|read_html|to_html|
|text|Local clipboard|read_clipboard|to_clipboard|
|binary|MS Excel|read_excel|to_excel|
|binary|Python Pickle Format|read_pickle|to_pickle|
|SQL|SQL|read_sql|to_sql|

In [ ]:
# 前半のダミーデータの読み込み
# DataFrame.head() は先頭の 5 件を表示
df_0 = pd.read_csv("./pandas練習用データ/ダミー個人データ_0.csv", encoding="shiftjis")
df_0

In [ ]:
# 後半のダミーデータの読み込み
# DataFrame.tail() は末尾の 5 件を表示
df_1 = pd.read_csv("./pandas練習用データ/ダミー個人データ_1.csv", encoding="shiftjis")
df_1

In [ ]:
df = pd.concat([df_0, df_1], ignore_index=True)
df

In [ ]:
# 都道府県名を元に都道府県コード列を加える
# まず都道府県コードデータを読み込む
df_pref_code = pd.read_csv("./pandas練習用データ/都道府県コード.csv", encoding="utf-8")
df_pref_code.head()

In [ ]:
# columns のラベルをキーにして連結するため
# ラベルを変更する
df_pref_code.columns = ["都道府県コード", "都道府県"]
df_pref_code.head()

In [ ]:
# 都道府県をキーに、都道府県コード列を追加
df = pd.merge(df, df_pref_code, on="都道府県", how="left")
df.head()

In [ ]:
# 欠損値がないか列ごとに確認
df.isna().any()

In [ ]:
# 同じ電話番号を持っている人がいないか確認 (True なら OK)
len(df["電話番号"]) == len(df["電話番号"].unique())

In [ ]:
# 同じ携帯 (番号) を持っている人がいないか確認
len(df["携帯"]) == len(df["携帯"].unique())

In [ ]:
# 都道府県のように重複がある場合は…
len(df["都道府県"]) == len(df["都道府県"].unique())

In [ ]:
# カレーの食べ方を正規表現を使って詳細化 (分割)
df["カレールーの位置"] = df["カレーの食べ方"].str.extract("^(.*)・(.*)$")[0]
df["カレーの攻め方"] = df["カレーの食べ方"].str.extract("^(.*)・(.*)$")[1]
df.head()

In [ ]:
# 年齢を元に世代に分類
bin = [10, 20, 30, 40, 50, 60, 70, 80, 100]
lab = ["10代", "20代", "30代", "40代", "50代", "60代", "70代", "80代以上"]

df["世代"] = pd.cut(df["年齢"], bins=bin, right=False, labels=lab)
df.head()

In [ ]:
# 世代の確認
df[df["世代"] == "10代"].head()

In [ ]:
# 世代の確認
df[df["世代"] == "70代"].head()

In [ ]:
# Pandas は内部で Matplotlib をインポートしており
# DataFrame から直接グラフをプロットすることができる
df_age = df["年齢"]
df_age.hist()

In [ ]:
# 別項目のヒストグラムを表示
df["キャリア"].hist()

In [ ]:
# Matplotlib はフォントを追加しなければ日本語を表示できない
# 今回は日本語名称を英字名称に置き換える
# まずは文字化けして読めない項目を確認する
df["キャリア"].unique()

In [ ]:
# それぞれ置き換えを実施
df.loc[df["キャリア"] == "ソフトバンク", ["キャリア"]] = "SoftBank"
df.loc[df["キャリア"] == "ドコモ", ["キャリア"]] = "NTT DoCoMo"
df.loc[df["キャリア"] == "ツーカー", ["キャリア"]] = "au"  # ツーカーは au ということにする(^^)/
df["キャリア"].hist()

In [ ]:
# 世代毎に携帯電話を契約しているかどうかの確率を設定し
# それをもとにデータを修正する関数を定義する
import numpy as np

def is_uncontracted(gen_series: pd.Series, cont_prob: dict) -> pd.Series:
    
    gen_arr = gen_series.to_numpy()  # pandas.Series を numpy.ndarray に変換
    buf = []
    for gen in gen_arr:
        if np.random.rand() <= 1-prob[gen]:
            buf.append(True)
        else:
            buf.append(False)
    return pd.Series(buf)

In [ ]:
# 元のデータをコピー
df_ = df.copy()

# 契約しているかどうかの確率を設定
prob = {"10代": 0.70, "20代": 0.98, "30代": 0.98, "40代": 0.97, "50代": 0.95, "60代": 0.89, "70代": 0.78, "80代以上": 0.65}

In [ ]:
# 定義した関数を実行して新しい列とする
df_["契約見直し"] = is_uncontracted(df_["世代"], prob)
df_.head()

In [ ]:
# 関数実行結果を元にデータを修正
df_.loc[df_["契約見直し"], ["キャリア"]] = "no_contract"
df_.loc[df_["契約見直し"], ["携帯"]] = np.nan  # ひとまず NaN にしておく
df_[df_["キャリア"]=="no_contract"].head()

In [ ]:
# 携帯を契約していない人の電話番号 (携帯) を NaN に変換したので
# もう一度欠損値を確認してみる
df_.isna().any()

In [ ]:
# 欠損値を変換
df_["携帯"] = df_["携帯"].fillna("-")
df_["携帯"].isna().any()

In [ ]:
# ヒストグラムでも確認
df_["キャリア"].hist()

In [ ]:
# キャリアのグループごとに size (この場合は人数) を表示
df_.groupby("キャリア").size()

In [ ]:
# ピボットテーブルでも確認
pd.pivot_table(df_, index="キャリア", columns="世代", values=["名前"], aggfunc="count")

In [ ]:
# 保存用のデータのために列の順序の並び替え＆必要な列のみ抽出
# 不要な列項目を削除するだけなら df_.drop(columns=["label_1", "label_2, ..."])
df_processed = df_[["名前", "ふりがな", "アドレス", "性別", "年齢", "世代", "都道府県", "都道府県コード", "電話番号", "携帯", "キャリア", "カレールーの位置", "カレーの攻め方"]]
df_processed.head()

In [ ]:
# 加工済みのデータを保存
df_processed.to_csv("./pandas練習用データ/加工済みデータ.csv", encoding="utf-8")

---
## データの前処理_その2  

[StatLib---Datasets Archive](http://lib.stat.cmu.edu/datasets/) で公開されているデータを `./pandas練習用データ/cadata.csv` に加工＆保存しています。それを読み込んでデータを眺めてみます。  

このデータセットは、アメリカの国勢調査で得られたカリフォルニア州の住宅価格などの情報を集約したものです。  

各 column の説明  

|columns|descriptions|
|-|-|
|median_house_value|Median house value for households within a block (measured in US Dollars)|  
|median_income|Median income for households within a block of houses (measured in tens of thousands of US Dollars)|  
|housing_median_age|Median age of a house within a block; a lower number is a newer building|  
|total_rooms|Total number of rooms within a block|  
|total_bedrooms|Total number of bedrooms within a block|  
|population|Total number of people residing within a block|  
|households|Total number of households, a group of people residing within a home unit, for a block|  
|latitude|A measure of how far north a house is; a higher value is farther north|  
|longitude|A measure of how far west a house is; a higher value is farther west|


In [ ]:
df_housing = pd.read_csv("./pandas練習用データ/cadata.csv", encoding="utf-8")  #　ヘッダー行がない場合は header=None が必要
df_housing.head()

In [ ]:
type(df_housing)

### データの冒頭/末尾部分を表示する head()/tail() メソッド  

DataFrame 型が持つ `head()` メソッドで、全てを表示すると長くなるような大量のデータの冒頭の部分だけを表示します。同様に `tail()` メソッドでデータの末尾の部分だけを表示させることも可能です。  

DataFrame 型は pandas で標準的に使われます。縦横に表のような形式でデータが並んでおり、一番上の行には見出しが、一番左の列にはインデックスが格納されます。  

In [ ]:
df_housing.head()

In [ ]:
df_housing.tail()

### DataFrame の行数列数を表示する shape 属性 (プロパティ)  

DataFrame 型の持つ `shape` プロパティによって行数と列数を確認することができます。  

In [ ]:
df_housing.shape

### 簡単な統計情報を見る describe() メソッド  

DataFrame 型の持つ `describe()` メソッドによって基本的な統計量を可視化することができます。それぞれの行は上から順に、データ数、平均、標準偏差、最小値、25％点、50％点（メジアン）、75％点、最大値を表します。  

In [ ]:
df_housing.describe()

### 個々の統計情報の表示  

DataFrame 型の持つ `count(), mean(), std(), var()` などのメソッドで特定の統計情報のみ表示させることもできます。  

In [ ]:
# 分散の表示
df_housing.var()

### 相関係数の表示
各列間の相関係数を表示する `corr()` メソッドも便利です。

In [ ]:
# 相関係数の一覧を表示
df_housing.corr()

## データの抽出

### 1 次元にデータが並んだ Series 型  

DataFrame 型が 2 次元的な広がりを持つ表であるのに対し、Series は 1 次元にデータが並んだものです。DataFrame は各行にインデックスが付与されていますが、Series 型も同様に各要素にインデックスが付与されています。上記で見たように、DataFrame から1列を抽出すると Series 型になります。  

In [ ]:
# "longitude"列を抽出する
sr_housing_longitude = df_housing["longitude"]
sr_housing_longitude.head()

In [ ]:
# Series型であることを確認
type(sr_housing_longitude)

### DataFrame 型から複数の列を取り出す  

DataFrame 型から複数の列を取り出す場合は、`[ ]` に抽出する列の見出しのリストを渡します。  

In [ ]:
df_housing_coordinate = df_housing[["longitude", "latitude"]]
df_housing_coordinate.head()

In [ ]:
type(df_housing_coordinate)

### 条件による行の抽出  

次の例では住宅の築年数の中央値がを 2 年以内の行を抽出します。  

1. `df_sample_housing["housing_median_age"]` で `housing_median_age` 列を抽出する  
1. `df_sample_housinge["housing_median_age"] <= 2` で `housing_median_age` 列の値が `2` を以下の行をインデックスと真偽値で表した `Series` を得る  
1. `df_sample_housing[df_sample_housing["housing_median_age"] <= 2]` で `df_sample_housing` から条件に対応した (真偽値 `True` の行の) データを抽出する  


In [ ]:
df_housing[df_housing["housing_median_age"] <= 2]

緯度経度でプロットしてみます。  
何となくカリフォルニア州っぽく見えます。

In [ ]:
df_housing.plot.scatter(x="longitude", y="latitude")

In [ ]:
df_housing["median_house_value"].hist()

In [ ]:
df_housing.plot.scatter(x="median_income", y="median_house_value")

"median_house_value" <= 500000 の値はすべて 500000 になっていそうなので、除外して再度プロットしてみます。   

In [ ]:
df_housing_cutoff = df_housing[df_housing["median_house_value"]<500000]

In [ ]:
df_housing_cutoff["median_house_value"].hist()

In [ ]:
df_housing_cutoff.plot.scatter(x="median_income", y="median_house_value")

このように、データをプロットしてみて修正を繰り返す作業を行う場合は、seaborn の `pairplot()` を利用するといいかもしれません。  

[seaborn.pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html)  

## 補足資料  

この資料による説明は以上です。さらに詳しく知りたい場合は、  

- [pandas: Python Data Analysis Library](http://pandas.pydata.org/)  
- [11. Pandas 入門 — ディープラーニング入門：Chainer チュートリアル](https://tutorials.chainer.org/ja/src/11_Introduction_to_Pandas_ja.html)  
- [Pandas入門 - DeepAge](https://deepage.net/features/pandas/)  

などを参照してください。